# Install Library

In [1]:
!pip install selenium webdriver-manager



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/43.8 kB ? eta -:--:--
   ---------------------------------------- 43.8/43.8 kB 2.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/63.8 kB ? eta -:--:--
   ---------------------------------------- 63.8/63.8 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.8.0
    Uninstalling typing_extensions-4.8.0:
      Successfully uninstalled typing_extensions-4.8.0
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0


# Main

In [5]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from urllib.parse import urlparse, parse_qs
import time
import pandas as pd

# === SETUP CHROME DALAM MODE HEADLESS ===
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Tidak menampilkan browser
options.add_argument("--disable-gpu")  # Tambahan supaya stabil di Windows
options.add_argument("--window-size=1920,1080")  # Set ukuran layar virtual

# Inisialisasi browser dengan opsi headless
driver = webdriver.Chrome(options=options)

# Buka halaman awal
start_url = "https://www.hp.com/id-id/shop/laptops-tablets.html"
driver.get(start_url)
time.sleep(3)

# Deteksi jumlah halaman terakhir
try:
    last_page_elem = driver.find_element(By.CSS_SELECTOR, "li.Last_list-page a")
    last_page_url = last_page_elem.get_attribute("href")
    parsed_url = urlparse(last_page_url)
    last_page = int(parse_qs(parsed_url.query)['p'][0])
    print(f"🔢 Halaman terakhir terdeteksi: {last_page}")
except:
    last_page = 1
    print("⚠️ Tidak bisa mendeteksi halaman terakhir, default ke 1")

produk_list = []

# Loop melalui semua halaman
for page_num in range(1, last_page + 1):
    print(f"\n🔄 Scraping Page {page_num}")
    driver.get(f"https://www.hp.com/id-id/shop/laptops-tablets.html?p={page_num}")
    time.sleep(5)

    items = driver.find_elements(By.CSS_SELECTOR, "li.item.product")

    for i, item in enumerate(items, 1):
        try:
            product_name = item.find_element(By.CSS_SELECTOR, "h2.plp-h2-title").text.strip()
        except:
            product_name = "Product is not found"

        try:
            rating = item.find_element(By.CSS_SELECTOR, ".bv_averageRating_component_container .bv_text")\
                         .get_attribute("textContent").strip()
        except:
            rating = "N/A"

        try:
            total_reviews = item.find_element(By.CSS_SELECTOR, ".bv_numReviews_component_container .bv_text")\
                                 .get_attribute("textContent").strip().replace("(", "").replace(")", "")
        except:
            total_reviews = "N/A"

        try:
            price_element = item.find_element(By.CSS_SELECTOR, "span.price-wrapper[data-price-amount]")
            price = price_element.get_attribute("data-price-amount")
        except:
            price = "N/A"

        print(f"{i}. {product_name}; Rating: {rating}; Reviews: {total_reviews}; Price: {price}")
        produk_list.append({
            "Page": page_num,
            "No": i,
            "Product Name": product_name,
            "Rating": rating,
            "Total Reviews": total_reviews,
            "Price": price
        })

    print("=" * 60)

# Simpan ke file CSV
df = pd.DataFrame(produk_list)
df.to_csv("daftar_laptop_hp.csv", index=False, encoding="utf-8-sig", sep=";")
print("\n✅ Data berhasil disimpan ke 'daftar_laptop_hp.csv'")

# Tutup browser
driver.quit()

🔢 Halaman terakhir terdeteksi: 13

🔄 Scraping Page 1
1. HP 14 inch Laptop 14s-dq5227TU, Silver; Rating: 4.2; Reviews: 942; Price: 8049000
2. HP 14 inch Laptop 14s-dq5156TU, Silver; Rating: 4.2; Reviews: 942; Price: 8449000
3. HP 14 inch Laptop 14s-dq5567TU, Silver; Rating: 4.2; Reviews: 942; Price: 5949000
4. HP 14 inch Laptop 14-em0019AU, Gold; Rating: 4.3; Reviews: 194; Price: 7899000
5. HP 14 inch Laptop 14-em0018AU, Silver; Rating: 4.3; Reviews: 194; Price: 7899000
6. HP 14 inch Laptop 14s-dq5227TU, Silver + HP P24v G5 FHD MNTR INDO; Rating: 4.2; Reviews: 942; Price: 9979000
7. HP 15.6 inch Laptop 15-fd0031TX, Silver; Rating: 4.4; Reviews: 676; Price: 9999000
8. HP 14 inch Laptop 14s-dq5155TU, Silver; Rating: 4.2; Reviews: 942; Price: 10249000
9. HP 14 inch Laptop 14s-dq5155TU, Silver + POLY BW 3220 USB-C HS USB-C/A; Rating: 4.2; Reviews: 942; Price: 11024000
10. HP Pavilion Aero 13.3 inch Laptop 13-bg0222AU, Blue + HP Smart Tank 585 Wi Fi All-in-One Printer; Rating: 4.3; Reviews: 